# Description

This is an automation script to categorize raw data captured in each experiment.

This script can be divided into four parts:

- Stage 0: User Settings: Modify settings to fit your experiment data.
- Stage 1: Collect metadata.
- Stage 2: Check files to make sure the numbers of each type of files are reasonable.
- Stage 3: Categorize files in chronological order, and organize with experiment names.

Before you go to stage 3, make sure you have already uploaded (manually or using py-script) all the raw data to database into specific directories, and removed the redundant files manually.

In [1]:
import os
import yaml
import shutil

# ===================== User Settings =====================
dates = [
    # "2024-03-19",
    # "2024-03-20",
    # "2024-05-14",
    # "2024-05-20"
    "2024-06-18",
    "2024-06-19",
    "2024-06-20",
    "2024-06-21",
]

target_folders = [
    "mi2log",
    "mi2log_xml",
    "server_pcap",
    "client_pcap",
]

common_folders = [
    "gps",
    "sync",
]

# ===================== Utils =====================
def makedir(dirpath):
    if not os.path.isdir(dirpath):
        print('    makedir:', dirpath)
        os.makedirs(dirpath)

def savecopy(filepath, targetdir, filename):
    # filename can be different from basename of filepath, can be used to rename a file.
    makedir(targetdir)
    print("    cp -p", filepath, os.path.join(targetdir, filename))
    shutil.copy2(filepath, os.path.join(targetdir, filename))  # reserve the last modified datetime and other metadata

def savemove(filepath, targetdir, filename):
    # filename can be different from basename of filepath, can be used to rename a file.
    makedir(targetdir)
    print("    mv", filepath, os.path.join(targetdir, filename))
    shutil.move(filepath, os.path.join(targetdir, filename))

# def delete(filepath):
#     if not os.path.exists(filepath):
#         print("rm: '{}': No such file or directory".format(filepath))
#         return
#     print("rm", filepath)
#     os.remove(filepath)

# def movedir(dirpath, targetdir, dirname):
#     # dirname can be different from basename of dirpath, can be used to rename a directory.
#     makedir(targetdir)
#     print("mv", dirpath, os.path.join(targetdir, dirname))
#     shutil.move(dirpath, os.path.join(targetdir, dirname))

# def copydir(dirpath, targetdir, dirname):
#     # dirname can be different from basename of dirpath, can be used to rename a directory.
#     makedir(targetdir)
#     print("copy", dirpath, os.path.join(targetdir, dirname))
#     shutil.copytree(dirpath, os.path.join(targetdir, dirname))

def deletedir_empty(dirpath):
    if not os.path.exists(dirpath):
        print("rmdir: '{}': No such file or directory".format(dirpath))
        return
    print("rmdir", dirpath)
    os.rmdir(dirpath)

# def deletedir_nonempty(dirpath):
#     if not os.path.exists(dirpath):
#         print("rm -rf: '{}': No such file or directory".format(dirpath))
#         return
#     print("rm -rf", dirpath)
#     shutil.rmtree(dirpath, ignore_errors=True)

def get_folder_size(folder_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # 跳過符號連結
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    unit_mapping = {0: 'bytes', 1: 'KB', 2: 'MB', 3: 'GB', 4: 'TB', 5: 'PB'}
    cnt = 0
    while total_size > 1000:
        total_size /= 1000
        cnt += 1
    unit = unit_mapping[cnt]
    total_size = f'{round(total_size, 2)} {unit}'
    return total_size

def get_file_size(file_path):
    if os.path.isfile(file_path):
        size = os.path.getsize(file_path)
        unit_mapping = {0: 'bytes', 1: 'KB', 2: 'MB', 3: 'GB', 4: 'TB', 5: 'PB'}
        cnt = 0
        while size > 1000:
            size /= 1000
            cnt += 1
        unit = unit_mapping[cnt]
        size = f'{round(size, 2)} {unit}'
        return size
    else:
        raise ValueError(f"{file_path} is not a valid file.")

def get_size(path):
    if os.path.isdir(path):
        return get_folder_size(path)
    elif os.path.isfile(path):
        return get_file_size(path)
    else:
        raise ValueError(f"{path} is not a valid file or directory.")

# Stage 1: Collect Metadata

In [2]:
dates = sorted(dates)
with open(os.path.join(os.getcwd(), "db_path.txt"), "r") as f:
        PATHS_TO_DATABASE = [s.strip() for s in f.readlines()]
    
date_paths = []

for date in dates:
    not_found_in_database = []
    
    for db_path in PATHS_TO_DATABASE:
        date_path = os.path.join(db_path, date)
        if os.path.isdir(date_path):
            date_paths.append(date_path)
        else:
            not_found_in_database.append(date_path)
    
    if len(not_found_in_database) == len(PATHS_TO_DATABASE):
        error_message = "[Errno 2] No such file or directory:\n"
        for date_path in not_found_in_database:
            error_message += "  '{}'\n".format(date_path)
        raise FileNotFoundError(error_message.strip())

metadatas = {}

for i, date_path in enumerate(date_paths):
    print('------------------')
    print(os.path.basename(date_path), ': ', os.path.dirname(date_path), sep='')
    
    yml_file = os.path.join(date_path, os.path.basename(date_path) + '.yml')
    with open(yml_file, 'r') as f:
        data = yaml.safe_load(f)
    
    metadatas[date_path] = [0]
    
    for j, (exp_name, exp) in enumerate(data.items()):
        if j > 0: print('  ------------------')
        print(' ', exp_name, '->', 'Skip:', exp['skip'], '|', 'UE:', exp['ue'], '|', 'Laptop:', exp['laptop'], '|', 'Route:', exp['route'])
        print('   ', exp['devices'])
        if exp['skip']:
            continue
        trips = []
        for key, value in exp['ods'].items():
            print('     ', key, value)
            if key > 0: trips.append(f'#{key:02d}')
        
        metadatas[date_path].append({
            'exp_name': exp_name,
            'ue': exp['ue'],
            'laptop': exp['laptop'],
            'route': exp['route'],
            'devices': list(exp['devices']),
            'trips': trips[:]
        })
        metadatas[date_path][0] += len(trips)
    
    if i == len(date_paths) - 1: print('------------------')

------------------
2024-06-18: /home/wmnlab/E/database
  UDP_Bandlock_9S_Phone_B_1 -> Skip: False | UE: Phone | Laptop: lpt3 | Route: B
    {'sm00': 'All', 'sm01': 'All', 'sm02': 'B1B3', 'sm03': 'B1B7', 'sm04': 'B1B8', 'sm05': 'B3B7', 'sm06': 'B3B8', 'sm07': 'B7B8', 'sm08': 'LTE'}
      0 ['起站', '迄站', '出發時間(hh:mm)', '備註']
      1 ['南港展覽館', '頂埔', '15:53', []]
      2 ['頂埔', '南港展覽館', '17:12', []]
  ------------------
  UDP_Bandlock_9S_Phone_B_2 -> Skip: False | UE: Phone | Laptop: lpt3 | Route: B
    {'sm00': 'All', 'sm01': 'All', 'sm02': 'B1B3', 'sm03': 'B1B7', 'sm04': 'B1B8', 'sm05': 'B3B7', 'sm06': 'B3B8', 'sm07': 'B7B8', 'sm08': 'LTE'}
      0 ['起站', '迄站', '出發時間(hh:mm)', '備註']
      1 ['南港展覽館', '頂埔', '18:11', []]
      2 ['頂埔', '南港展覽館', '19:19', []]
------------------
2024-06-19: /home/wmnlab/E/database
  UDP_Bandlock_9S_Phone_G_1 -> Skip: False | UE: Phone | Laptop: lpt3 | Route: G
    {'sm00': 'All', 'sm01': 'All', 'sm02': 'B1B3', 'sm03': 'B1B7', 'sm04': 'B1B8', 'sm05': 'B3B7', 'sm

# Stage 2: Check Files

檢查資料的數量是否正確

- mi2log, mi2log_xml: 機台數量*總趟次
- client_pcap: 機台數量*總趟次 (Bilink)
- server_pcap: 機台數量*總趟次\*2 or 1 (Uplink/Downlink, Bi-link)

In [3]:
for i, (date_path, metadata) in enumerate(metadatas.items()):
    print('-----------------------------')
    print(date_path, '({})'.format(metadata[0]), get_size(date_path))
    
    for j, exp in enumerate(metadata[1:]):
        if j > 0: print('  -----------------------------')
        print(' ', exp['exp_name'], '->', 'UE:', exp['ue'], '|', 'Laptop:', exp['laptop'], '|', 'Route:', exp['route'])
        print('   ', len(exp['devices']), exp['devices'])
        print('     ', len(exp['trips']), exp['trips'])
    
    for target_folder in target_folders:
        filenames = sorted(os.listdir(os.path.join(date_path, target_folder)))
        print('  ------------------ target folder')
        print(' ', target_folder, '({})'.format(len(filenames)), get_size(os.path.join(date_path, target_folder)))
        for filename in filenames:
            print('   ', filename, get_size(os.path.join(date_path, target_folder, filename)))
    
    for common_folder in common_folders:
        if not os.path.isdir(os.path.join(date_path, common_folder)):
            print('  ------------------ common folder')
            print(' ', common_folder, 'does not exist!')
            continue
        filenames = sorted(os.listdir(os.path.join(date_path, common_folder)))
        print('  ------------------ common folder')
        print(' ', common_folder, '({})'.format(len(filenames)), get_size(os.path.join(date_path, common_folder)))
        for filename in filenames:
            print('   ', filename, get_size(os.path.join(date_path, common_folder, filename)))
    
    if i == len(date_paths) - 1: print('-----------------------------')

-----------------------------
/home/wmnlab/E/database/2024-06-18 (4) 70.43 GB
  UDP_Bandlock_9S_Phone_B_1 -> UE: Phone | Laptop: lpt3 | Route: B
    9 ['sm00', 'sm01', 'sm02', 'sm03', 'sm04', 'sm05', 'sm06', 'sm07', 'sm08']
      2 ['#01', '#02']
  -----------------------------
  UDP_Bandlock_9S_Phone_B_2 -> UE: Phone | Laptop: lpt3 | Route: B
    9 ['sm00', 'sm01', 'sm02', 'sm03', 'sm04', 'sm05', 'sm06', 'sm07', 'sm08']
      2 ['#01', '#02']
  ------------------ target folder
  mi2log (36) 314.19 MB
    diag_log_sm00_2024-06-18_15-53-03.mi2log 10.46 MB
    diag_log_sm00_2024-06-18_17-12-17.mi2log 10.14 MB
    diag_log_sm00_2024-06-18_18-11-17.mi2log 10.71 MB
    diag_log_sm00_2024-06-18_19-19-26.mi2log 9.83 MB
    diag_log_sm01_2024-06-18_15-53-03.mi2log 10.51 MB
    diag_log_sm01_2024-06-18_17-12-17.mi2log 10.92 MB
    diag_log_sm01_2024-06-18_18-11-17.mi2log 10.44 MB
    diag_log_sm01_2024-06-18_19-19-26.mi2log 10.72 MB
    diag_log_sm02_2024-06-18_15-53-03.mi2log 5.07 MB
    diag_

# Stage 3: Categorize

In [4]:
for i, (date_path, metadata) in enumerate(metadatas.items()):
    print('------------------')
    print(date_path, '({})'.format(metadata[0]))
    
    for j, exp in enumerate(metadata[1:]):
        if j > 0: print('  ------------------')
        print(' ', exp['exp_name'], '->', 'UE:', exp['ue'], '|', 'Laptop:', exp['laptop'], '|', 'Route:', exp['route'])
        print('   ', len(exp['devices']), exp['devices'])
        print('     ', len(exp['trips']), exp['trips'])
        for target_folder in target_folders:
            src_dirpath = os.path.join(date_path, target_folder)
            print('  ------------------')
            print(' ', target_folder, '({})'.format(len(os.listdir(src_dirpath))))
            for dev in exp['devices']:
                print('   ---', dev)
                filenames = sorted([s for s in os.listdir(src_dirpath) if dev in s])
                for trip in exp['trips']:
                    filename = filenames.pop(0)
                    dest_dirpath = os.path.join(date_path, exp['exp_name'], dev, trip, 'raw')
                    savemove(os.path.join(src_dirpath, filename), dest_dirpath, filename)
    
    if i == len(date_paths) - 1: print('------------------')

for i, (date_path, metadata) in enumerate(metadatas.items()):
    for target_folder in target_folders:
        src_dirpath = os.path.join(date_path, target_folder)
        deletedir_empty(src_dirpath)

------------------
/home/wmnlab/E/database/2024-06-18 (4)
  UDP_Bandlock_9S_Phone_B_1 -> UE: Phone | Laptop: lpt3 | Route: B
    9 ['sm00', 'sm01', 'sm02', 'sm03', 'sm04', 'sm05', 'sm06', 'sm07', 'sm08']
      2 ['#01', '#02']
  ------------------
  mi2log (36)
   --- sm00
    makedir: /home/wmnlab/E/database/2024-06-18/UDP_Bandlock_9S_Phone_B_1/sm00/#01/raw
    mv /home/wmnlab/E/database/2024-06-18/mi2log/diag_log_sm00_2024-06-18_15-53-03.mi2log /home/wmnlab/E/database/2024-06-18/UDP_Bandlock_9S_Phone_B_1/sm00/#01/raw/diag_log_sm00_2024-06-18_15-53-03.mi2log
    makedir: /home/wmnlab/E/database/2024-06-18/UDP_Bandlock_9S_Phone_B_1/sm00/#02/raw
    mv /home/wmnlab/E/database/2024-06-18/mi2log/diag_log_sm00_2024-06-18_17-12-17.mi2log /home/wmnlab/E/database/2024-06-18/UDP_Bandlock_9S_Phone_B_1/sm00/#02/raw/diag_log_sm00_2024-06-18_17-12-17.mi2log
   --- sm01
    makedir: /home/wmnlab/E/database/2024-06-18/UDP_Bandlock_9S_Phone_B_1/sm01/#01/raw
    mv /home/wmnlab/E/database/2024-06-18/m